In [1]:
// Some utility functions etc; stuff common to each day's puzzle

In [2]:
using System.IO;
using System.Text.RegularExpressions;
using SCG = System.Collections.Generic;

In [3]:
// Console output is repeated when this util file is imported in other
// notebooks, so skip these tests to keep the imports clean
private static bool RUN_TESTS = false;

In [4]:
/// <summary>
/// Load a given day's puzzle input
/// </summary>
string[] LoadPuzzleInput(int year, int day)
{
    var filePath = $"Day{day}.txt";
    Console.WriteLine($"Loading puzzle file: {filePath}");
    
    var lines = File.ReadAllLines(filePath);

    Console.WriteLine($"Total lines: {lines.Length}");
    var maxLineLength = lines.Select(line => line.Length).Max();
    Console.WriteLine($"Max line length: {maxLineLength}");
    Console.WriteLine();

    return lines;
}

In [5]:
/// <summary>
/// Display a sample of the puzzle input lines
/// </summary>
void WriteLines<T>(IEnumerable<T> lines, int maxRows = 5, int maxCols = 500)
{
    foreach (var line in lines.Take(maxRows))
    {
        var lineStr = line.ToString();
        lineStr = (lineStr.Length - maxCols) switch {
            > 0 => lineStr.Substring(0, maxCols),
            _ => lineStr
        };
        Console.WriteLine(lineStr);
    }
}

In [6]:
/// <summary>
/// Exception thrown when <c>Ensure</c> check does not match
/// </summary>
class PuzzleFailureException(string message) : Exception(message) {}

/// <summary>
/// Ensure puzzle answer is correct
/// </summary>
void Ensure<T>(T expected, T answer) where T : IComparable<T>
{
    if (!expected.Equals(answer)) {
        throw new PuzzleFailureException($"Expected puzzle answer to equal {expected}, was {answer}");
    }
}

In [7]:
// Loosely based on:
// https://github.com/dotnet/csharplang/discussions/388#discussioncomment-67632

/// <summary>Array deconstructor helper</summary>
public static void Deconstruct<T>(this T[] array, out T item1, out T item2, out T item3, out T item4)
{
    item1 = GetValueOrDefault(array, 0);
    item2 = GetValueOrDefault(array, 1);
    item3 = GetValueOrDefault(array, 2);
    item4 = GetValueOrDefault(array, 3);
}

/// <summary>Array deconstructor helper</summary>
public static void Deconstruct<T>(this T[] array, out T item1, out T item2, out T item3)
{
    (item1, item2, item3, _) = array;
}

/// <summary>Array deconstructor helper</summary>
public static void Deconstruct<T>(this T[] array, out T item1, out T item2)
{
    (item1, item2, _, _) = array;
}

/// <summary>Array deconstructor helper</summary>
public static void Deconstruct<T>(this T[] array, out T item1)
{
    (item1, _, _, _) = array;
}

private static T GetValueOrDefault<T>(T[] array, int i) => (array.Length - i) switch {
    > 0 => array[i],
    _ => default(T)
};

In [8]:
using System.Text.RegularExpressions;

/// <summary>
/// Regex deconstructor helper - assign values corresponding to gc[i].Value, or null if no group
/// </summary>
public static void Deconstruct(this GroupCollection gc, out string a, out string b, out string c, out string d)
{
    var values = Enumerable.Range(0, 4).Select(i => i < gc.Count ? gc[i].Value : null).ToArray();
    a = values[0];
    b = values[1];
    c = values[2];
    d = values[3];
}

/// <summary>
/// Regex deconstructor helper - assign values corresponding to gc[i].Value, or null if no group
/// </summary>
public static void Deconstruct(this GroupCollection gc, out string a, out string b, out string c)
{
    (a, b, c, _) = gc;
}

In [9]:
/// <summary>
/// Separate <c>source</c> into multiple groups
/// </summary>
/// <remarks>Useful when the puzzle input contains multiple sections</remarks>
public static IEnumerable<string[]> SeparateBy(this IEnumerable<string> source, Func<string, bool> separator)
{
    ArgumentNullException.ThrowIfNull(source);
    ArgumentNullException.ThrowIfNull(separator);

    List<string> current = new();
    foreach (var line in source)
    {
        if (separator(line))
        {
            yield return current.ToArray();
            current = new();
            continue;
        }
        
        current.Add(line);
    }
    yield return current.ToArray();
}

In [10]:
// Waiting for dotnet-interactive to update to .NET 9...
// https://github.com/dotnet/interactive/issues/3760

if (Environment.Version.Major >= 9) {
    throw new Exception("We don't need the following extensions anymore!");
}

/// <summary>
/// Enumerate a collection alongside its index
/// </summary>
/// <remarks>Analogue of System.Linq.Enumerable.Index in .NET 9</remarks>
public static IEnumerable<(int Index, T Item)> Index<T>(this IEnumerable<T> source)
{
    var i = 0;
    foreach (var item in source)
    {
        yield return (i++, item);
    }
}

In [11]:
// Points and Grids
/////

record struct Point(int X, int Y)
{
    public override string ToString() => $"({X}, {Y})";
    
    public static Point operator +(Point a, Point b) => (a.X + b.X, a.Y + b.Y);
    public static Point operator -(Point a, Point b) => a + (-b);
    public static Point operator -(Point p) => (-p.X, -p.Y);
    public static Point operator *(Point p, int x) => (p.X * x, p.Y * x);

    public static implicit operator Point((int x, int y) xy) => new(xy.x, xy.y);
}

static readonly Point Up = (0, -1);
static readonly Point Down = -Up;
static readonly Point Left = (-1, 0);
static readonly Point Right = -Left;

Dictionary<Point, Point> RightTurns = new()
{
    { Up, Right },
    { Right, Down },
    { Down, Left },
    { Left, Up }
};
Point TurnRight(Point p) => RightTurns[p];

In [12]:
// Tests
if (RUN_TESTS) 
{
    Point p = new(1, 1);
    Console.WriteLine(p);

    foreach (var testPoint in new[] {Up, Down, Left, Right})
    {
        Console.WriteLine(p + testPoint);
    }

    Console.WriteLine(p * 5);
}

In [13]:
class CharGrid(string[] inputLines)
{
    string[] _inputLines = inputLines;

    /// <summary>
    /// Maximum addressible row
    /// </summary>
    public int Rows { get; } = inputLines.Length;

    /// <summary>
    /// Maximum addressible column
    /// </summary>
    public int Cols { get; } = inputLines[0].Length;

    /// <summary>
    /// Character at corresponding point
    /// </summary>
    public char this[Point p]
    {
        get => _inputLines[p.Y][p.X];
        set
        {
            var lineBuffer = _inputLines[p.Y].ToCharArray();
            lineBuffer[p.X] = value;
            _inputLines[p.Y] = new string(lineBuffer);
        }
    }

    /// <summary>
    /// Enumerate all points and characters in the grid
    /// </summary>
    /// <remarks>Useful for searching for specific features</remarks>
    public IEnumerable<(Point point, char ch)> Enumerate()
    {
        var q = from y in Enumerable.Range(0, Rows)
                from x in Enumerable.Range(0, Cols)
                let p = new Point(x, y)
                select (p, this[p]);
        return q;
    }

    /// <summary>
    /// Is <c>p</c> a valid point within the grid?
    /// </summary>
    public bool IsValid(Point p) => p switch
    {
        (< 0, _) => false,
        (_, < 0) => false,
        (var x, _) when x >= Cols => false,
        (_, var y) when y >= Rows => false,
        _ => true
    };

    /// <summary>
    /// Clone current grid
    /// </summary>
    public CharGrid Clone() => new([.._inputLines]);

    /// <summary>
    /// Render the grid for printing to console
    /// </summary>
    public string Render() => String.Join("\n", _inputLines);
}

In [14]:
static Dictionary<char, int> DigitLookup = new()
{
    { '0', 0 },
    { '1', 1 },
    { '2', 2 },
    { '3', 3 },
    { '4', 4 },
    { '5', 5 },
    { '6', 6 },
    { '7', 7 },
    { '8', 8 },
    { '9', 9 }
};

/// <summary>
/// Convert a number character into the numeric value. A common puzzle requirement.
/// </summary>
public static int ToInt(this char ch) => DigitLookup[ch];

In [15]:
/// <summary>
/// Helper to ensure loops always terminate
/// </summary>
class SafetyLimit
{
    int _attempts = 0;

    public void EnsureBelow(int limit)
    {
        if (++_attempts == limit)
        {
            throw new Exception($"Safety limit of {limit} exceeded.");
        }
    }
}

In [16]:
// Tests

if (RUN_TESTS)
{
    string[] testInputLines = [
        "ab",
        "cd"
    ];

    CharGrid grid = new(testInputLines);

    Point p = new(0, 0);
    Console.WriteLine(grid[p]);
    Console.WriteLine(grid[p + Right]);
    Console.WriteLine(grid[p + Down]);
    Console.WriteLine(grid[p + Down + Right]);

    Console.WriteLine(grid.IsValid(p));
    Console.WriteLine(grid.IsValid(p + Right));
    Console.WriteLine(grid.IsValid(p + Right + Right));
}

In [17]:
static IEnumerable<LinkedListNode<T>> WalkNodes<T>(this LinkedList<T> list)
{
    var node = list.First;
    while (node != null)
    {
        yield return node;
        node = node.Next;
    }
}

In [18]:
/// <summary>
/// Breadth-First Search, another common puzzle algorithm.
/// </summary>
IEnumerable<T> BFS<T>(T startNode, Func<T, IEnumerable<T>> getNeighbours)
{
    HashSet<T> visited = new();
    
    Queue<T> queue = new();
    queue.Enqueue(startNode);

    while (queue.TryDequeue(out var node))
    {
        if (visited.Contains(node)) continue;

        yield return node;
        
        visited.Add(node);

        foreach (var neighbour in getNeighbours(node))
        {
            queue.Enqueue(neighbour);
        }
    }
}

In [19]:
using static System.Globalization.CultureInfo;

/// <summary>
/// Solve a system of linear equations via Gaussian elimination
/// <summary>
/// <paramref name="matrix">Matrix of linear equations expressed in rows, eg: Ax + By = C</paramref>
/// <remarks>https://en.wikipedia.org/wiki/Gaussian_elimination</remarks>
void GaussianElim(decimal[][] matrix)
{
    var rows = matrix.Length;
    var cols = matrix[0].Length;

    var pivotRow = 0;
    var pivotCol = 0;

    var range = (int minInc, int maxEx) => Enumerable.Range(minInc, maxEx - minInc);

    void swapRows(int i, int j)
    {
        var tmp = matrix[i];
        matrix[i] = matrix[j];
        matrix[j] = tmp;
    }

    decimal[] scale(int row, decimal scale) => matrix[row].Select(col => col * scale).ToArray();

    void add(int row, decimal[] otherRow)
    {
        matrix[row] = matrix[row].Zip(otherRow, (a, b) => a + b).ToArray();
    }

    while (pivotRow < rows && pivotCol < cols)
    {
        var rowMax = range(pivotRow, rows).OrderByDescending(i => Math.Abs(matrix[i][pivotCol])).First();

        if (matrix[rowMax][pivotCol] == 0)
        {
            pivotRow++;
        }
        else
        {
            swapRows(pivotRow, rowMax);

            // Now that we have our pivot row / colum. We can zero-out the pivot
            // column for all rows below the pivot row
            foreach (var rowBelow in range(pivotRow + 1, rows))
            {
                var ratio = matrix[rowBelow][pivotCol] / matrix[pivotRow][pivotCol];
                add(row: rowBelow, scale(pivotRow, -ratio));
                // Mitigation for precision issues
                matrix[rowBelow][pivotCol] = 0;
            }

            pivotRow++;
            pivotCol++;
        }
    }

    // We are now in row-echelon form. Now to reduce...

    // Note that due to small losses of precision during the reduction stage,
    // values for zero and one can be slightly off, eg zero is
    // 0.0000000000000000000000000001. Mitigations have been added below.

    foreach (var row in range(0, rows))
    {
        var pivot = Array.FindIndex(matrix[row], col => Math.Abs(col) > 0.01m);
        if (pivot != -1)
        {
            matrix[row] = scale(row, 1m / matrix[row][pivot]);
            // Mitigation for precision issues
            matrix[row][pivot] = 1;

            foreach (var aboveRow in range(0, row))
            {
                add(aboveRow, scale(row, -matrix[aboveRow][pivot]));
                // Mitigation for precision issues
                matrix[aboveRow][pivot] = 0;
            }
        }
    }
}

void Render(decimal[][] matrix)
{
    foreach (var row in matrix)
    {
        // var rowNums = string.Join(" ", row.Select(col => col.ToString("0.##", CurrentCulture)));
        var rowNums = string.Join(" ", row);        
        Console.WriteLine($"| {rowNums} |");
    }
    Console.WriteLine();
}

In [20]:
// Tests

if (RUN_TESTS) 
{
    decimal[][] gaussianTestInput = [
        [2, 1, -1, 8],
        [-3, -1, 2, -11],
        [-2, 1, 2, -3]
    ];

    Render(gaussianTestInput);

    GaussianElim(gaussianTestInput);

    Render(gaussianTestInput);
}

In [21]:
// Not 100% tested, but hopefully this allows long-running calculations to be
// stopped without having to restart

/// <summary>
/// Check whether the cancel button has been pressed in the notebook UI. 
/// </summary>
static bool TaskCancelled => Microsoft.DotNet.Interactive.KernelInvocationContext.Current.CancellationToken.IsCancellationRequested;

In [22]:
/// <summary>
/// Used in <see>ShortestPath. For a given node + lowest cost, return updated costs for neighbouring nodes</see>
/// </summary> 
delegate IEnumerable<(T node, TCost cost)> NextNodeFunc<T, TCost>(T node, TCost cost);

/// <summary>
/// Dijkstra's shortest path algorithm
/// </summary>
/// <param name="start">Node from which to start (assume 0 cost)</param>
/// <param name="nextNodeFunc">For a given node, return reachable nodes + costs</param>
IEnumerable<(T node, TCost cost)> ShortestPath<T, TCost>(T start, NextNodeFunc<T, TCost> nextNodeFunc)
{
    PriorityQueue<T, TCost> queue = new(); 
    HashSet<T> visited = new();

    queue.Enqueue(start, default);
    while (queue.TryDequeue(out var node, out var cost))
    {
        if (visited.Contains(node))
        {
            // Already processed (at lowest cost)
            continue;
        }

        yield return ((node, cost));
        
        visited.Add(node);

        foreach (var (nextNode, nextCost) in nextNodeFunc(node, cost))
        {
            queue.Enqueue(nextNode, nextCost);
        }
    }
}

const int MAX_COST = int.MaxValue;

In [23]:
// Utils to parse numbers (or other regexes) from string inputs

/// <summary>
/// Parse all matches of a regex (eg: all numbers)
/// </summary>
static IEnumerable<string> ParseAll(this IEnumerable<string> lines, string regex) 
{
    var q = from line in lines
            from match in Regex.Matches(line, regex)
            select match.ToString();
    return q;
}

/// <summary>
/// Parse all matches of a regex (eg: all numbers)
/// </summary>
static IEnumerable<string> ParseAll(this string line, string regex) => ParseAll([line], regex);